In [1]:
import os
import openai

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from git import Repo

In [4]:
from pathlib import Path

In [5]:
pwd

'/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator'

In [7]:
PATH_TO_BLOG_REPO = Path("/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/.git")

In [9]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [10]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [11]:
PATH_TO_CONTENT

PosixPath('/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/content')

In [12]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [14]:
def update_blog(commit_message='Updates blog'):
    # GitPython -- Repo locations
    repo = Repo(PATH_TO_BLOG_REPO)
    # git add .
    repo.git.add(all=True)
    # git commit -m "updates blog"
    repo.index.commit(commit_message)
    # git push
    origin = repo.remote(name='origin')
    origin.push()

In [15]:
random_text_string = "kdsaflkjiewoasvantluat"

In [16]:
with open(PATH_TO_BLOG/"index.html", 'w') as f:
    f.write(random_text_string)

In [17]:
update_blog()

In [21]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image= Path(cover_image)
    
    # auto file naming utility
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WRITE A NEW HTML FILE
        with open(path_to_new_content,"w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            # OpenAI --->> Completion GPT -->> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exists, please check again your name! Aborting!")